In [26]:
import pandas as pd
import configparser
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from import_dfs import Datenbank

In [155]:
'''Datentypen
    cities: Stadtdaten
    tech_data: Technologiedaten
    improve_data: Verbesserungsfaktoren
    rf_pv_basis: Referenzstrekcen Basis
    rf_pv_pod: Referenzstrecken Pod
    rf_gv_basis: Referenzstrecken Güterverkehr Basis
    rf_gv_pod: Referenzstrecken Güterverkehr Pod
    rf_pv_routes: Strecken Personenverkehr 
    rf_gv_routes: Strecken Güterverkehr
'''

db = Datenbank()

In [156]:
df = db.rf_pv_basis.copy()  # init

def find_unique_modes(df):
    '''
    Bekommt eine Referenzstreckentabelle und ermittelt alle unique modes.
    Achtung: Für den Güterverehr muss die Anzahl an Schleifen durchläufen angepasst werden.
    '''
    x = set()
    for i in range(1,7):
        tmp = df.loc[:, f'mode_{i}'].unique().to_list()
        x.update(tmp)
        x.discard(np.nan)
    return x


def lower_and_underscores(l):
    '''
    Recives a list and lowers all words. In a second step it although replaces spaces with underscores.
    '''
    return [word.lower().replace(" ", '_').replace('-', '_') for word in l]


unique_modes = find_unique_modes(df)
unique_modes_fitted = lower_and_underscores(unique_modes)
unique_modes_dict = dict(zip(unique_modes, unique_modes_fitted))
unique_modes_dict[np.nan] = np.nan

f = lambda x: unique_modes_dict[x]  # Funktion um Werte aus dict in df zu schreiben
db.rf_pv_basis.loc[:, [f'mode_{i}' for i in range(1,7)]] = db.rf_pv_basis.loc[:, [f'mode_{i}' for i in range(1,7)]].applymap(f)  # Anwenden der Funktion

In [157]:
# Transformieren de DataFrames, so dass Verkehrsmittel in Spalten stehen 

df = db.rf_pv_basis.copy()  # init

df_new = pd.DataFrame(0, index=df.index, columns=unique_modes_fitted)

result = pd.concat([df, df_new], axis=1, sort=False)

def transform_df(row):
    '''
    Nimmt eine row und schreit die Gesamtkilometer je Verkehrmittel in das zur Spalte gehörige Feld.
    '''
    for i in range(1,7):
        mode = row.loc[f'mode_{i}']
        if mode not in [np.nan, np.NaN, 'nan']:
            row.loc[mode] += row.loc[f'mode_length_{i}']
        else:
            continue
    return row


result = result.apply(lambda row: transform_df(row), axis=1)

result = result.drop(columns=[f'mode_{i}' for i in range(1,7)] + [f'mode_length_{i}' for i in range(1,7)])

db.rf_pv_basis = result

In [221]:
# Berechnung der Emissionen je Kilometer

tech_data = db.tech_data.copy()  # init
rf_pv_basis = db.rf_pv_basis.copy()  # init

per_trip_costs = ["flugzeug", 'bus', 'tram', 'u_bahn']  # In der Literatur nur per Trip angegeben

fitted_index_lvl_0 = lower_and_underscores(tech_data.index.levels[0])  # modes werden angepasst
categorize = tech_data.index.levels[1].to_list()  # Kategorien, in denen die Bewertung vorgenommen wird
categorize.remove('verfuegbarkeit')
categorize.remove('auslastung')

tech_data.index.set_levels(fitted_index_lvl_0, level=0, inplace=True)  # Umbenennen der Kategorien
tech_data = pd.DataFrame(tech_data.loc[:, 'klassisch'])  # Ausprobieren für Szenario Klassisch muss angepasst werden

for mode in fitted_index_lvl_0:
        for cat in categorize:
            if mode in rf_pv_basis.columns.to_list() and not (cat == "kosten" and mode in per_trip_costs or cat == 'unfallrisiko'): # Standardfall
                rf_pv_basis.loc[:,f'{mode}_{cat}'] = (rf_pv_basis.loc[:, mode] / 1000) * tech_data.loc[(mode, cat),:].values[0]
            elif mode in rf_pv_basis.columns.to_list() and (cat == "kosten" and mode in per_trip_costs):  # Kosten werden per Trip berechnet
                rf_pv_basis.loc[:,f'{mode}_{cat}'] = rf_pv_basis.apply(lambda x: tech_data.loc[(mode, cat),:].values[0] if x.loc[mode] != 0 else 0, axis=1)
            elif mode in rf_pv_basis.columns.to_list() and cat == 'unfallrisiko':  # Unfallrisiko ist auf 1 Mrd. km bezogen
                rf_pv_basis.loc[:,f'{mode}_{cat}'] = rf_pv_basis.loc[:, mode] * (tech_data.loc[(mode, cat),:].values[0] / 1000000000000)  # Trillion

                
def sum_emissions(df):
    for cat in categorize:
        df.loc[:, f'{cat}'] = df.loc[:,[f'{mode}_{cat}' for mode in unique_modes_fitted]].sum(axis=1)
    return df
            
rf_pv_basis = sum_emissions(rf_pv_basis)

rf_pv_basis.loc[:, 'main_mode'] = rf_pv_basis.loc[:, unique_modes_fitted].idxmax(axis=1)  # find mode with highest length percentage

rf_pv_basis.loc[:, ['wartezeit', 'fahrtzeit', 'length']] = rf_pv_basis.loc[:, ['wartezeit', 'fahrtzeit', 'length']].round(0).astype(int)
rf_pv_basis.loc[:, 'kosten'] = rf_pv_basis.loc[:, 'kosten'].round(2) 
rf_pv_basis.loc[:, ['energieverbrauch', 'thg', 'nox', 'pm']] = rf_pv_basis.loc[:, ['energieverbrauch', 'thg', 'nox', 'pm']].round(4)

ausgabe = rf_pv_basis.loc[:, ['main_mode', 'wartezeit', 'fahrtzeit', 'length', 'kosten', 'energieverbrauch', 'thg', 'nox', 'pm', 'unfallrisiko'] ]
ausgabe

,main_mode,wartezeit,fahrtzeit,length,kosten,energieverbrauch,thg,nox,pm,unfallrisiko
index,,,,,,,,,,
0.0,zu_fuss,0,30,2500,0.00,0.0000,0.0000,0.0000,0.0000,3.550000e-08
1.0,fahrrad,0,8,2600,0.00,0.0000,0.0000,0.0000,0.0000,2.418000e-08
2.0,miv,0,16,3600,0.96,6.0800,444.8000,1.0880,1.2800,1.272000e-08
3.0,bus,1,18,3790,3.88,3.1280,228.9300,0.5648,0.0033,9.732000e-09
4.0,e_bike,0,7,2600,0.00,0.0650,9.5186,0.0078,0.0000,2.418000e-08
...,...,...,...,...,...,...,...,...,...,...
216.0,zug_fernverkehr,199,466,721340,125.38,382.2300,27596.2500,12.5540,2.7711,6.144580e-07
217.0,flugzeug,90,170,549060,139.92,856.0600,104677.9600,264.4996,53.1504,3.504880e-07
218.0,flugzeug,109,188,540100,133.30,799.9040,100513.8600,256.0608,34.2281,2.728000e-07
